In [131]:
import numpy as np 
import matplotlib.pyplot as plt
import gym
import ray.rllib.agents.ppo as ppo
import random
import jdc
import pandas as pd

from gym import Env, spaces
import time

#get a matrix of distances between locations
df = pd.read_csv('dist_vologda_matrix.csv', sep=',')
df = df.iloc[:, 1:]

#some parameters
working_hours = 12
truck_speed = 60

df = df / 1000
df

,АЗС №35400 Кичменгский Городок,АЗС №35401 Великий Устюг-1,АЗС №35402 Великий Устюг-2,АЗС №35403 Дунай,АЗС №35404 Нюксеница,АЗС №35405 Борисово-Судское,АЗС №35406 Журавлево,АЗС №35407 Грязовец-1,АЗС №35408 Грязовец-2,АЗС №35409 Савино,...,АЗС №35454 Октябрьский пр.-1 (Вологда),АЗС №35455 Октябрьский пр.-2 (Вологда),АЗС №35456 Маяковская,АЗС №35457 Краснодонцев,АЗС №35458 Северсталь,АЗС №35459 Олимпийская,АЗС №35460 Северное шоссе-1,АЗС №35461 Северное шоссе-2,АЗС № 35462 Аксеново,Нефтебаза
0,0.0000,99.2272,101.8513,139.2135,154.2522,777.2635,102.4328,520.5020,517.8863,266.7014,...,610.3886,610.3028,607.3283,609.0579,611.0852,611.0605,604.2684,605.7555,502.9826,477.9325
1,99.0684,0.0000,4.9465,134.5201,149.5589,748.0318,5.5280,491.2703,488.6546,241.7276,...,581.1570,581.0711,578.0967,579.8263,581.8535,581.8288,575.0368,576.5238,473.7510,448.7008
2,101.8148,5.0688,0.0000,137.2665,152.3053,750.7781,3.0302,494.0166,491.4010,244.4739,...,583.9033,583.8174,580.8430,582.5726,584.5999,584.5752,577.7831,579.2702,476.4973,451.4472
3,139.2135,134.6791,137.3031,0.0000,15.0961,613.5690,137.8846,356.8075,354.1918,107.2648,...,446.6942,446.6083,443.6339,445.3635,447.3908,447.3660,440.5740,442.0610,339.2882,314.2380
4,154.2522,149.7178,152.3419,15.0961,0.0000,613.8947,152.9234,357.1332,354.5176,107.5905,...,447.0199,446.9341,443.9596,445.6892,447.7165,447.6918,440.8997,442.3868,339.6139,314.5638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,611.7861,582.6174,585.2414,447.9957,448.3214,191.2351,585.8229,181.8523,179.2367,345.2112,...,10.1276,10.0417,7.2967,3.3719,10.8241,0.0000,6.1623,4.6720,107.2118,142.1894
60,605.7962,576.6275,579.2515,442.0057,442.3315,185.0682,579.8330,175.8624,173.2467,339.2213,...,6.8726,6.7867,3.8122,4.8070,7.5691,6.8095,0.0000,1.5045,101.2218,136.1995
61,607.1467,577.9779,580.6020,443.3562,443.6820,186.4186,581.1835,177.2129,174.5972,340.5717,...,8.2230,8.1372,5.1627,3.5396,8.9196,5.5422,1.5229,0.0000,102.5723,137.5500
62,504.6641,475.4954,478.1194,340.8736,341.1994,274.3706,478.7009,74.7303,72.1146,238.0892,...,107.4957,107.4099,104.4354,106.1650,108.1923,108.1676,101.3755,102.8626,0.0000,35.0674


In [132]:
#remove locations, which cannot be served in 12 hours (driver can't get there and come back in 12 hours)
to_remove = ((2 * (df.values[:,-1] / truck_speed) < working_hours) == False).nonzero()[0]
df = df.drop(df.index[to_remove])
df = df.drop(df.columns[to_remove], axis=1)
df = df.reset_index(drop=True)
N = len(df.values[0])
df

,АЗС №35403 Дунай,АЗС №35404 Нюксеница,АЗС №35405 Борисово-Судское,АЗС №35407 Грязовец-1,АЗС №35408 Грязовец-2,АЗС №35409 Савино,АЗС №35410 Сазоново-1,АЗС №35411 Сазоново-2,АЗС №35412 Вытегра,АЗС №35413 Липник,...,АЗС №35454 Октябрьский пр.-1 (Вологда),АЗС №35455 Октябрьский пр.-2 (Вологда),АЗС №35456 Маяковская,АЗС №35457 Краснодонцев,АЗС №35458 Северсталь,АЗС №35459 Олимпийская,АЗС №35460 Северное шоссе-1,АЗС №35461 Северное шоссе-2,АЗС № 35462 Аксеново,Нефтебаза
0,0.0000,15.0961,613.5690,356.8075,354.1918,107.2648,619.0077,618.5201,629.3182,472.6696,...,446.6942,446.6083,443.6339,445.3635,447.3908,447.3660,440.5740,442.0610,339.2882,314.2380
1,15.0961,0.0000,613.8947,357.1332,354.5176,107.5905,619.3335,618.8458,629.6440,472.9954,...,447.0199,446.9341,443.9596,445.6892,447.7165,447.6918,440.8997,442.3868,339.6139,314.5638
2,615.1980,615.5238,0.0000,349.0547,346.4390,512.4135,142.9597,142.4720,165.8965,193.2966,...,187.4480,187.3622,184.7850,188.3816,183.6881,191.1772,185.1426,186.6296,274.4141,309.3918
3,356.1259,356.4516,349.7479,0.0000,3.7018,253.3414,355.1867,354.6990,373.5276,216.8790,...,182.8731,182.7873,179.8128,181.5424,183.5697,183.5450,176.7529,178.2400,75.4671,45.6714
4,353.5507,353.8764,347.1727,3.7422,0.0000,250.7662,352.6115,352.1238,370.9524,214.3038,...,180.2979,180.2120,177.2376,178.9672,180.9945,180.9698,174.1777,175.6648,72.8919,43.0962
5,110.2445,110.5703,510.6206,253.8591,251.2434,0.0000,516.0593,515.5716,526.3698,369.7212,...,343.7458,343.6599,340.6854,342.4151,344.4423,344.4176,337.6256,339.1126,236.3397,211.2896
6,620.5994,620.9251,142.9494,354.4560,351.8403,517.8149,0.0000,0.4877,505.5423,348.8937,...,192.8494,192.7635,190.1863,193.7829,189.0895,196.5785,190.5439,192.0309,279.8155,314.7931
7,620.1117,620.4374,142.4617,353.9683,351.3527,517.3272,0.4877,0.0000,505.0546,348.4060,...,192.3617,192.2759,189.6987,193.2952,188.6018,196.0909,190.0562,191.5433,279.3278,314.3054
8,630.8056,631.1314,166.0027,373.2181,370.6024,528.0211,504.5004,504.0127,0.0000,156.6990,...,332.1868,332.1010,329.1265,330.8561,332.8834,332.8587,326.0666,327.5537,345.1153,329.7586
9,474.1483,474.4741,342.4044,216.5608,213.9452,371.3638,347.8431,347.3554,156.8593,0.0000,...,175.5295,175.4437,172.4692,174.1988,176.2261,176.2014,169.4093,170.8964,188.4580,173.1013


In [133]:
#custom gym environment for our problem
class TransportScape(Env):
    def __init__(self, env_config):
        self.num_of_trucks = N - 2
        self.num_of_locations = N - 1
        self.h = working_hours #working hours of each truck driver
        self.speed = truck_speed
        self.distances = df.values #an array, containing distances between locations
    
        self.action_space = spaces.MultiDiscrete([self.num_of_trucks, self.num_of_locations])
    
        self.observation_space = spaces.Dict(
        {
            #x_i - shows each truck's location
            'truck location': spaces.MultiDiscrete(np.full(self.num_of_trucks, self.num_of_locations)),
            #p_j - shows whether a truck is assigned to this particular location or not
            'assignment': spaces.MultiBinary(self.num_of_locations), 
            #u_i - shows whether a particular truck was used before
            'truck usage': spaces.MultiBinary(self.num_of_trucks),
            #delta_i - shows how much time each truck has left until the end of the work day
            'time left': spaces.Box(low=np.zeros(self.num_of_trucks), high=np.full(self.num_of_trucks, self.h), dtype=np.float64)
        })
        self.reset()
        
    def reset(self):
        self.state = {'truck location': np.full(self.num_of_trucks, self.num_of_locations),
                      'assignment': np.zeros(self.num_of_locations),
                      'truck usage': np.zeros(self.num_of_trucks),
                      'time left': np.full(self.num_of_trucks, self.h)}
        self.reward = 0
        self.done = False
        return self.state 

In [134]:
%%add_to TransportScape

    def step(self, action):
        i, j = action[0], action[1]
        
        truck_loc = self.state['truck location']
        assignment = self.state['assignment']
        truck_use = self.state['truck usage']
        time_left = self.state['time left']
        
        #if all locations have trucks assigned to them - end episode
        if np.all(self.state['assignment'] == 1):
            self.done = True
        #if a truck is already assigned to this location - penalty
        if assignment[j] == 1:
            self.reward -= 100
        #if no time left - penalty
        if (time_left[i] - self.distances[truck_loc[i], j] / self.speed - self.distances[truck_loc[-1], j] / self.speed) < 0:
            self.reward -= 100
        #else - assign truck to a new location
        else:
            self.reward = 0
            #if needs to use an unused before truck - penalty
            if truck_use[i] == 0:
                self.reward -= 100
                truck_use[i] = 1
            assignment[i] = 1
            self.reward -= self.distances[truck_loc[i], j]
            time_left[i] -= self.distances[truck_loc[i], j] / self.speed
            truck_loc[i] = j
        
        self.state['truck location'] = truck_loc
        self.state['assignment'] = assignment
        self.state['truck usage'] = truck_use
        self.state['time left'] = time_left
        
        return self.state, self.reward, self.done, {}

In [135]:
config = ppo.DEFAULT_CONFIG.copy()
config["env_config"] = {}
config["num_workers"] = 1
config["framework"] = "torch"
config['num_cpus_per_worker'] = 0  # This avoids running out of resources in the notebook environment when this cell is re-executed
#config["disable_env_checking"] = True
#config

In [136]:
#trainer = ppo.PPOTrainer(config, TransportScape)

In [137]:
from stable_baselines3 import PPO

env = TransportScape(config)
model = PPO("MultiInputPolicy", env, verbose=1)
model.learn(total_timesteps=1000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


RuntimeError: Class values must be smaller than num_classes.

In [ ]:
obs = env.reset()
result = []
done = False
while not done:
    action, _states = model.predict(obs, deterministic=True)
    print(i)
    print(action)
    obs, reward, done, info = env.step(action)
    if done:
        obs = env.reset()
        break
env.close()

In [ ]:
#plot reward max